In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os

# --- 1. CONFIGURATION ---
BATCH_SIZE = 1
EPOCHS = 15  # Increased epochs because training from scratch takes longer
LEARNING_RATE = 1e-3  # Increased LR slightly for training from scratch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATASET_PATH = "/home/aditya/Desktop/Xyaa/Brain_tumour_classification/archive/Augmented_Training"

print(f"Using device: {DEVICE}")

# --- 2. DATASET DEFINITION ---
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.image_paths = []
        self.labels = []

        print(f"Found classes: {self.classes}")

        for label, category in enumerate(self.classes):
            category_path = os.path.join(root_dir, category)
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')): 
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            return torch.zeros(3, 256, 256), label

# --- 3. TRANSFORMS & LOADERS ---
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # Normalized for standard training stability
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
])

if os.path.exists(DATASET_PATH):
    full_dataset = BrainTumorDataset(DATASET_PATH, transform=transform)
    NUM_CLASSES = len(full_dataset.classes)
    
    train_size = int(0.9 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    print(f"✅ Data Loaded. Train: {len(train_dataset)} | Val: {len(val_dataset)} | Classes: {NUM_CLASSES}")
else:
    print(f"❌ Error: Path not found at {DATASET_PATH}")
    exit()

# --- 4. CUSTOM EFFICIENTNET DEFINITION ---

# Swish Activation
def swish(x):
    return x * torch.sigmoid(x)

# Squeeze-and-Excitation Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=4):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(in_channels, in_channels // reduction)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels)

    def forward(self, x):
        batch, channels, _, _ = x.size()
        y = x.view(batch, channels, -1).mean(dim=2)
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y))
        y = y.view(batch, channels, 1, 1)
        return x * y

# MBConv Block
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, expansion, kernel_size, stride):
        super(MBConv, self).__init__()
        hidden_dim = in_channels * expansion
        self.use_residual = (stride == 1 and in_channels == out_channels)

        self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        
        self.dwconv = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride,
                                padding=kernel_size//2, groups=hidden_dim, bias=False)
        self.bn2 = nn.BatchNorm2d(hidden_dim)

        self.se = SEBlock(hidden_dim)
        self.conv2 = nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = swish(self.bn1(self.conv1(x)))
        out = swish(self.bn2(self.dwconv(out)))
        out = self.se(out)
        out = self.bn3(self.conv2(out))
        if self.use_residual:
            out += x
        return out

class CustomEfficientNetB3(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientNetB3, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

        config = [
            (1,  16, 3, 1, 3), 
            (6,  24, 3, 2, 3), 
            (6,  40, 5, 2, 5), 
            (6,  80, 3, 2, 5), 
            (6, 112, 5, 1, 5), 
            (6, 160, 5, 2, 5), 
            (6, 224, 3, 1, 5) 
        ]
        
        in_channels = 32
        self.blocks = []
        for expansion, out_channels, kernel_size, stride, num_blocks in config:
            layers = [MBConv(in_channels, out_channels, expansion, kernel_size, stride)]
            for _ in range(num_blocks - 1):
                layers.append(MBConv(out_channels, out_channels, expansion, kernel_size, 1))
            self.blocks.append(nn.Sequential(*layers))
            in_channels = out_channels
        self.blocks = nn.Sequential(*self.blocks)

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(224, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        return x

# Initialize Custom Model
model = CustomEfficientNetB3(num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- 5. TRAINING LOOP ---
print("\nStarting Training (From Scratch)...")
best_val_acc = 0.0

for epoch in range(EPOCHS):
    # Train
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_acc = 100 * val_correct / val_total

    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  -> Saved new best model.")

print("\nTraining Complete.")

Using device: cpu
Found classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
✅ Data Loaded. Train: 30844 | Val: 3428 | Classes: 4

Starting Training (From Scratch)...


In [1]:
# Evaluation


import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             classification_report, confusion_matrix, roc_curve, auc)
from sklearn.preprocessing import label_binarize
import os

# ==========================================
# 1. CONFIGURATION
# ==========================================
# UPDATE THIS PATH TO YOUR TEST DATASET
TEST_DATA_PATH = r"D:\Projects\brain_tumor_project\Dataset\Testing" 

MODEL_PATH = r"D:\Projects\brain_tumor_project\src\components\models\best_model.pth"
OUTPUT_DIR = "evaluation_results"
BATCH_SIZE = 32
NUM_CLASSES = 4
CLASS_LABELS = ["glioma", "meningioma", "pituitary", "notumour"] 

# Create output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. MODEL DEFINITION
# ==========================================
class BrainTumorEfficientNet(nn.Module):
    def __init__(self, num_classes=4):
        super(BrainTumorEfficientNet, self).__init__()
        self.model = models.efficientnet_b3(weights=None)
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# ==========================================
# 3. DATA LOADING
# ==========================================
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

try:
    test_dataset = ImageFolder(root=TEST_DATA_PATH, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    print(f"Data Loaded. Classes: {test_dataset.classes}")
except Exception as e:
    print(f"Error: {e}")
    exit()

# ==========================================
# 4. INFERENCE LOOP
# ==========================================
def get_predictions():
    model = BrainTumorEfficientNet(num_classes=NUM_CLASSES).to(device)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.eval()

    y_true = []
    y_pred = []
    y_probs = []

    print("Running Inference...")
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    return np.array(y_true), np.array(y_pred), np.array(y_probs)

y_true, y_pred, y_probs = get_predictions()

# ==========================================
# 5. GLOBAL METRICS
# ==========================================
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted')
rec = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("\n" + "="*40)
print(" GLOBAL PERFORMANCE METRICS")
print("="*40)
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")
print("-"*40)

# ==========================================
# 6. PLOT: PER-LABEL ACCURACY
# ==========================================
# Calculate accuracy for each class (Diagonal of Confusion Matrix / Total per row)
cm = confusion_matrix(y_true, y_pred)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)

plt.figure(figsize=(10, 6))
bars = plt.bar(test_dataset.classes, per_class_accuracy, color=['#4287f5', '#f54242', '#32a852', '#f5a142'])

plt.xlabel('Tumor Classes', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Per-Class Accuracy', fontsize=14)
plt.ylim(0, 1.1)

# Add value labels on top of bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.02, f"{yval:.2%}", ha='center', fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.7)
save_path = os.path.join(OUTPUT_DIR, 'per_class_accuracy.png')
plt.savefig(save_path)
print(f"Saved: {save_path}")
plt.close()

# ==========================================
# 7. PLOT: ROC CURVE (Multi-Class)
# ==========================================
y_true_bin = label_binarize(y_true, classes=range(NUM_CLASSES))

plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'orange']

for i, class_name in enumerate(test_dataset.classes): 
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=colors[i], lw=2,
             label=f'{class_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Random guess line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Analysis')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

save_path_roc = os.path.join(OUTPUT_DIR, 'roc_curve.png')
plt.savefig(save_path_roc)
print(f"Saved: {save_path_roc}")
plt.close()

print("\nEvaluation Complete.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "c:\Users\admin\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\admin\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\admin\anaconda3\Lib\site-pac

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "c:\Users\admin\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\admin\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\admin\anaconda3\Lib\site-pac

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "c:\Users\admin\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\admin\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\admin\anaconda3\Lib\site-pac

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\admin\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "c:\Users\admin\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\admin\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\admin\anaconda3\Lib\site-pac

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



Data Loaded. Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
Running Inference...

 GLOBAL PERFORMANCE METRICS
Accuracy : 0.9710
Precision: 0.9728
Recall   : 0.9710
F1 Score : 0.9706
----------------------------------------
Saved: evaluation_results\per_class_accuracy.png
Saved: evaluation_results\roc_curve.png

Evaluation Complete.


In [2]:
pip check


aext-assistant-server 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-core 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-panels 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-panels-server 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-project-filebrowser-server 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-shared 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-share-notebook-server 4.1.0 requires anaconda-cloud-auth, which is not installed.
aext-toolbox 4.1.0 requires anaconda-cloud-auth, which is not installed.
numba 0.60.0 has requirement numpy<2.1,>=1.22, but you have numpy 2.2.6.
streamlit 1.37.1 has requirement pillow<11,>=7.1.0, but you have pillow 12.1.0.
tensorflow 2.19.0 has requirement numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6.
Note: you may need to restart the kernel to use updated packages.
